In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std 

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std 

(tensor(-7.6999e-06), tensor(1.))

In [7]:
#export
def test_near_zero(a, tol=1e-3): 
    assert a.abs()<tol, f"Near zero: {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1 - x_train.std())

In [9]:
n, m = x_train.shape
c = y_train.max() + 1
n, m, c

(50000, 784, tensor(10))

### Basic architecture

In [10]:
nh = 50

In [11]:
# simplified kaiming init / he init
w1 = torch.randn(m, nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [13]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [14]:
def lin(x, w, b): 
    return x@w + b

In [15]:
t = lin(x_valid, w1, b1)

In [16]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0244), tensor(0.9777))

In [17]:
def relu(x): 
    return x.clamp_min(0.)

In [18]:
t = relu(lin(x_valid, w1, b1))

In [19]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3987), tensor(0.5795))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [20]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [21]:
w1.mean(),w1.std()

(tensor(-0.0003), tensor(0.0506))

In [22]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.4716), tensor(0.7451))

In [23]:
#export
from torch.nn import init

In [24]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [25]:
w1.mean(),w1.std()

(tensor(0.0002), tensor(0.0505))

In [26]:
t.mean(),t.std()

(tensor(0.5202), tensor(0.8086))

In [27]:
w1.shape

torch.Size([784, 50])

In [28]:
import torch.nn

In [29]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

In [30]:
# what if...?
def relu(x): 
    return x.clamp_min(0.) - 0.5

In [31]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0090), tensor(0.7980))

In [38]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [39]:
%timeit -n 10 _=model(x_valid)

3.55 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE


In [41]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing(,1), in order to use `mse`.  (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [42]:
#export
def mse(output, targ):
    return (output.squeeze(-1) - targ).pow(2).mean()

In [43]:
y_train, y_valid = y_train.float(), y_valid.float()

In [44]:
preds = model(x_train)

In [45]:
preds.shape

torch.Size([50000, 1])

In [46]:
mse(preds, y_train)

tensor(29.0414)

### Gradients and backward pass

In [63]:
def mse_grad(inp, targ):
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [64]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [65]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [66]:
def forward_and_backward(inp, targ):
    # forward pass
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    
    #loss
    loss = mse(out, targ)
    
    # backward pass
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [78]:
forward_and_backward(x_train, y_train)

In [79]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [80]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [81]:
def forward(inp, targ):
    # forward pass
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    
    #loss
    return mse(out, targ)

In [82]:
loss = forward(xt2, y_train)

In [83]:
loss.backward()

In [84]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

In [85]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g

In [89]:
class Lin():
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [90]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [94]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [95]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [96]:
%time loss = model(x_train, y_train)

CPU times: user 108 ms, sys: 12 ms, total: 120 ms
Wall time: 29.8 ms


In [97]:
%time model.backward()

CPU times: user 3.51 s, sys: 4.33 s, total: 7.84 s
Wall time: 2.03 s


In [98]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [99]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self):
        raise Exception('not implemented')
        
    def backward(self):
        self.bwd(self.out, *self.args)

In [100]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.) - 0.5
    
    def bwd(self, out, inp):
        inp.g = (inp>0).float() * out.g

In [101]:
class Lin(Module):
    def __init__(self, w, b): 
        self.w,self.b = w,b
        
    def forward(self, inp): 
        return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [102]:
class Mse(Module):
    def forward(self, inp, targ):
        return (inp.squeeze() - targ).pow(2).mean()
    
    def bwd(self, out, inp, targ):
        inp.g = 2*(inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [103]:
class Model(Module):
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [105]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [106]:
%time loss = model(x_train, y_train)

CPU times: user 72 ms, sys: 44 ms, total: 116 ms
Wall time: 29.3 ms


In [107]:
%time model.backward()

CPU times: user 240 ms, sys: 196 ms, total: 436 ms
Wall time: 110 ms


In [108]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [109]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [110]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [111]:
%time loss = model(x_train, y_train)

CPU times: user 96 ms, sys: 28 ms, total: 124 ms
Wall time: 31.3 ms


In [112]:
%time model.backward()

CPU times: user 252 ms, sys: 132 ms, total: 384 ms
Wall time: 96 ms


In [113]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [115]:
#export
from torch import nn

In [116]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: 
            x = l(x)
        return self.loss(x.squeeze(), targ)

In [117]:
model = Model(m, nh, 1)

In [118]:
%time loss = model(x_train, y_train)

CPU times: user 76 ms, sys: 16 ms, total: 92 ms
Wall time: 64.3 ms


In [119]:
%time loss.backward()

CPU times: user 124 ms, sys: 44 ms, total: 168 ms
Wall time: 65.8 ms


### Export

In [121]:
!python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
